In [1]:
import toml
import pathlib

import numpy as np
import torch

import pyworld as pw
import librosa
import cv2

from cmvc import *
from cmvc.utils.data.dataset import PairDataset

In [2]:
import IPython.display as display

In [3]:
dict_toml = toml.load(open('/home/jun/Documents/CMVC/cmvc/config.toml'))

image_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["image"])
voice_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["voice"])

In [4]:
uttr_path = pathlib.Path(dict_toml["path"]["dataset"]["voice"])

In [5]:
dataset = PairDataset(voice_path=voice_path, image_path=image_path, train=True)

In [6]:
eval_data = PairDataset(voice_path=voice_path, image_path=image_path, train=False)

In [35]:
tmp = eval_data.__getitem__(idx=0, k=1)
wav, img = tmp[0], tmp[1]

[162077]
/home/jun/Documents/CMVC/voice/eval/VCC2SM3/30013.pkl


In [8]:
wav.shape

torch.Size([1, 1, 36, 180])

In [9]:
voice_inv = dataset.voice_transform.inv_trans
voice_trans = dataset.voice_transform.__call__
image_inv = dataset.image_transform.inv_trans
image_trans = dataset.image_transform.__call__

In [10]:
uttr_path = uttr_path/"vcc2018_database_evaluation/vcc2018_evaluation"/"VCC2SF1"

In [34]:
wav = list(uttr_path.iterdir())[0]

In [12]:
display.Audio(wav)

In [16]:
x, fs = librosa.load(wav, sr=22050, dtype=np.float64)
mccs = librosa.feature.melspectrogram(y=x, sr=fs, n_mels=36)

In [13]:
#img = image_path / "000085.jpg"

In [14]:
#display.Image(img)

In [51]:
net = Net()

In [52]:
model_dir = pathlib.Path('/home/jun/Documents/CMVC/checkpoint')
checkpoint = torch.load(model_dir/(str(10).zfill(4)+".cpt"))

In [53]:
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [54]:
def eval_net(net, voice, image,
             device="cpu"):
    net.eval()
    
    x = voice.to(device)
    y = image.to(device)
    
    return net.forward(x, y)

In [19]:
"""
x, fs = librosa.load(wav, sr=22050, dtype=np.float64)
mccs = librosa.feature.melspectrogram(y=x, sr=fs, n_mels=36)
"""

'\nx, fs = librosa.load(wav, sr=22050, dtype=np.float64)\nmccs = librosa.feature.melspectrogram(y=x, sr=fs, n_mels=36)\n'

In [20]:
"""
xx = torch.tensor([[voice_trans(mccs)]])
yy = torch.tensor(image_trans(cv2.imread(str(img))))
"""

'\nxx = torch.tensor([[voice_trans(mccs)]])\nyy = torch.tensor(image_trans(cv2.imread(str(img))))\n'

In [55]:
xx = wav
yy = img

In [56]:
device = "cpu"
net.to(device)

Net(
  (ue): UttrEncoder(
    (uttr_enc_d1): CBGLayer(
      (conv1): Conv2d(1, 16, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
      (conv2): Conv2d(1, 16, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (uttr_enc_d2): CBGLayer(
      (conv1): Conv2d(16, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (conv2): Conv2d(16, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (uttr_enc_d3): CBGLayer(
      (conv1): Conv2d(32, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (conv2): Conv2d(32, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (bn1): BatchN

In [57]:
voice_inv(xx).min()

tensor(1.8227e-08)

In [24]:
xx.size()

torch.Size([1, 1, 36, 180])

In [26]:
#net.fe.face_encoder(yy)[0]

In [28]:
#net.fe(yy)

In [58]:
eval_net(net, xx, yy).min()

tensor(-0.0628, grad_fn=<MinBackward1>)

In [59]:
conversion_mccs_tensor = eval_data.voice_transform.inv_trans(eval_net(net, xx, yy)).squeeze(0).squeeze(0)

In [60]:
conversion_mccs = conversion_mccs_tensor.detach().numpy().copy()

In [61]:
conversion_mccs.shape

(36, 180)

In [40]:
mccs = xx.squeeze(0).squeeze(0).detach().numpy().copy()

In [62]:
def mccs_to_audio(mccs):
    mccs = mccs.astype("float32")
    #mccs += mccs.min()
    S = librosa.feature.inverse.mel_to_stft(mccs)
    y = librosa.griffinlim(S, dtype=np.float32)
    
    return y

In [63]:
y = mccs_to_audio(conversion_mccs)

KeyboardInterrupt: 

In [ ]:
import soundfile as sf

In [ ]:
sf.write("conv.wav", y, 22050, "PCM_24")